# Merging Datasets

## Table of Content

#### 1. Importing Data

#### 2. Checks on department and aisles dataset

#### 3. First Merging: Order with prior_basket

#### 4. Second Merging: Order with train_basket

#### 5. Merging first and second Merging

#### 6. Third Merging: Products

#### 7. Fourth Merging: department information

#### 8. Fifth Merging: aisle information

#### 9. Last cleaning

#### 10. Saving Data

In [1]:
# import libraries
import pandas as pd
import numpy as np
import os

In [2]:
pd.options.display.max_columns = None

## 1. Import Data

In [3]:
path = r'/Users/henning/Portfolio/Instacart_Basket'

In [4]:
df_orders = pd.read_pickle(os.path.join(path, 'Data', 'Prepared Data', 'orders_cleaned.pkl'))

In [5]:
df_prior_baskets = pd.read_pickle(os.path.join(path, 'Data', 'Prepared Data', 'prior_baskets_cleaned.pkl'))

In [6]:
df_train_baskets = pd.read_pickle(os.path.join(path, 'Data', 'Prepared Data', 'train_baskets_cleaned.pkl'))

In [7]:
df_products = pd.read_pickle(os.path.join(path, 'Data', 'Prepared Data', 'products_cleaned.pkl'))

In [8]:
df_department = pd.read_csv(os.path.join(path, 'Data', 'Original Data', 'departments.csv'))

In [9]:
df_aisles = pd.read_csv(os.path.join(path, 'Data', 'Original Data', 'aisles.csv'))

## 2. Small checks on department and aisles dataset

In [10]:
# changing IDs into string type
df_department['department_id'] = df_department['department_id'].astype('str')
df_aisles['aisle_id'] = df_aisles['aisle_id'].astype('str')

## 3. First Merging: Order with prior_basket

In [11]:
# filtering df_order to include only the previous orders.
df_orders_prior = df_orders[df_orders['eval_set'] == 'prior']

In [12]:
# deciding type of join by checking number of rows
df_orders_prior.shape

(3214874, 7)

In [13]:
df_prior_baskets.shape

(32434489, 3)

In [14]:
# combining dataframes
df_prior =  df_prior_baskets.merge(df_orders_prior, on = 'order_id', how='left', indicator = True)

In [15]:
df_prior['_merge'].value_counts()

_merge
both          32434489
left_only            0
right_only           0
Name: count, dtype: int64

In [16]:
# dropping _merge column for the next combination
df_prior = df_prior.drop(columns = ['_merge'])

## 4. Second Merging: Order with the train_basket

In [17]:
# filtering df_order to include only the recent orders.
df_orders_recent = df_orders[df_orders['eval_set'] == 'train']

In [18]:
# deciding type of join by checking number of rows
df_orders_recent.shape

(131209, 7)

In [19]:
df_train_baskets.shape

(1384617, 3)

In [20]:
# combining dataframes
df_train =  df_train_baskets.merge(df_orders_recent, on = 'order_id', how='left', indicator = True)

In [21]:
df_train['_merge'].value_counts()

_merge
both          1384617
left_only           0
right_only          0
Name: count, dtype: int64

In [22]:
# dropping _merge column for the next combination
df_train = df_train.drop(columns = ['_merge'])

## 5. Merging df_prior and df_recent to df_1

In [23]:
# Concatenating the Data
frames = [df_prior, df_train]
df_1 = pd.concat(frames, axis = 0)

In [24]:
df_1

,order_id,product_id,reordered,user_id,eval_set,order_number,order_weekday,order_hour_of_day,days_since_prior_order
0,2,33120,1,202279,prior,3,5,9,8.0
1,2,28985,1,202279,prior,3,5,9,8.0
2,2,9327,0,202279,prior,3,5,9,8.0
3,2,45918,1,202279,prior,3,5,9,8.0
4,2,30035,0,202279,prior,3,5,9,8.0
...,...,...,...,...,...,...,...,...,...
1384612,3421063,14233,1,169679,train,30,0,10,4.0
1384613,3421063,35548,1,169679,train,30,0,10,4.0
1384614,3421070,35951,1,139822,train,15,6,10,8.0
1384615,3421070,16953,1,139822,train,15,6,10,8.0


## 6. Third Merging: Products

In [25]:
# deciding type of join by checking number of rows
df_1.shape

(33819106, 9)

In [26]:
df_products.shape

(49688, 4)

In [27]:
# combining dataframes with left join to include all orders, but to exclude unordered products.
df_2 =  df_1.merge(df_products, on = 'product_id', how='left', indicator = True)

In [28]:
df_2['_merge'].value_counts()

_merge
both          33819106
left_only            0
right_only           0
Name: count, dtype: int64

In [29]:
# dropping _merge column for the next combination
df_2 = df_2.drop(columns = ['_merge'])

## 7. Fourth Merging: department information

In [30]:
# deciding type of join by checking number of rows
df_2.shape

(33819106, 12)

In [31]:
df_department.shape

(21, 2)

In [32]:
# combining dataframes
df_3 =  df_2.merge(df_department, on = 'department_id', how='left', indicator = True)

In [33]:
df_3['_merge'].value_counts()

_merge
both          33819106
left_only            0
right_only           0
Name: count, dtype: int64

In [34]:
# dropping _merge column for the next combination
df_3 = df_3.drop(columns = ['_merge'])

## 8. Fifth Merging: aisles information

In [35]:
# combining dataframes
df_instacart =  df_3.merge(df_aisles, on = 'aisle_id', how='left', indicator = True)

In [36]:
df_instacart['_merge'].value_counts()

_merge
both          33819106
left_only            0
right_only           0
Name: count, dtype: int64

In [37]:
# dropping _merge column for the next combination
df_instacart = df_instacart.drop(columns = ['_merge'])

## 9. Removing ID columns

In [38]:
# because ID columns are only necessary for combining the data sets,
# they can now be removed as we don't need them for the upcoming analysis.
# Only the User_id stays, as it is the only way to identify a specific customer.
df_instacart = df_instacart.drop(columns = ['order_id', 'product_id', 'aisle_id', 'department_id'])

In [39]:
# renaming user_id column for better understanding and readability
df_instacart.rename(columns = {'user_id' : 'customer'}, inplace = True)

In [40]:
# final check on data
df_instacart.sample(3)

,reordered,customer,eval_set,order_number,order_weekday,order_hour_of_day,days_since_prior_order,product_name,department,aisle
22946831,0,182183,prior,10,0,16,7.0,Organic Whole Wheat Penne Rigate,dry goods pasta,dry pasta
5801763,0,190182,prior,4,1,12,10.0,Sweet Red Grape Tomatoes,produce,fresh vegetables
29391343,1,58919,prior,31,5,8,6.0,Organic Black Beans,canned goods,canned meals beans


## 10. Exporting merged Data Set

In [41]:
df_instacart.to_pickle(os.path.join(path, 'Data','Prepared Data', 'instacart.pkl'))